## Feast Client with RBAC

## Feast Kubernetes RBAC Authorization

Feast **Role-Based Access Control (RBAC)** in Kubernetes relies on a **service account** for authentication. This applies both **within a Kubernetes pod** and for **external clients** accessing Feast

In this example, Feast will automatically retrieve the Kubernetes ServiceAccount token from pod path:
```
/var/run/secrets/kubernetes.io/serviceaccount/token
```
This means:
- No manual configuration is needed inside a pod.
- The token is mounted automatically and used for authentication.
- Developer?User just need create the binding with role and service account accordingly.

For more details, refer to the user guide: [Kubernetes RBAC Authorization](https://docs.feast.dev/master/getting-started/components/authz_manager#kubernetes-rbac-authorization). 


###  Feature Store settings
**The Operator create client ConfigMap** containing the `feature_store.yaml `settings. We can retrieve it save it feature_repo folder.

In [20]:
!kubectl get configmap feast-sample-kubernetes-auth-client -n feast -o jsonpath='{.data.feature_store\.yaml}' > client/feature_repo/feature_store.yaml
!cat  client/feature_repo/feature_store.yaml

project: feast_rbac
provider: local
offline_store:
    host: feast-sample-kubernetes-auth-offline.feast.svc.cluster.local
    type: remote
    port: 80
online_store:
    path: http://feast-sample-kubernetes-auth-online.feast.svc.cluster.local:80
    type: remote
registry:
    path: feast-sample-kubernetes-auth-registry.feast.svc.cluster.local:80
    registry_type: remote
auth:
    type: kubernetes
entity_key_serialization_version: 3


**Create ConfigMap From Feature Repository**  
We need feature_repo inside the container. let's create configmap from `feature_repo` contains the feature repository files, including `feature-store.yaml` and `test.py`. It will be mounted as a volume in the deployment for the client examples to test the script.

In [21]:
!kubectl delete configmap client-feature-repo-config --ignore-not-found -n feast 
!kubectl create configmap client-feature-repo-config --from-file=client/feature_repo -n feast

configmap/client-feature-repo-config created


### Test Read-Only Feast User 

**Step 1: Deploy read-only user, we are using `serviceAccountName  feast-user-sa` in deployment.**


In [22]:
# Create the deployment 
!cat client/readonly_user_deployment.yaml
!kubectl apply -f "client/readonly_user_deployment.yaml"


apiVersion: apps/v1
kind: Deployment
metadata:
  name: client-readonly-user
  namespace: feast
  labels:
    app: client-user
spec:
  replicas: 1
  selector:
    matchLabels:
      app: client-user
  template:
    metadata:
      labels:
        app: client-user
    spec:
      serviceAccountName: feast-user-sa
      containers:
        - name: client-user-container
          image: quay.io/feastdev/feature-server:latest
          imagePullPolicy: Always
          command: ["sleep", "infinity"]
          volumeMounts:
            - name: client-feature-repo-config
              mountPath: /opt/app-root/src
      volumes:
        - name: client-feature-repo-config
          configMap:
            name: client-feature-repo-config
deployment.apps/client-readonly-user created


**Step 2: Run test.py script for client-readonly-user, readonly-user can only read or query all objects.**

In [23]:
#Run test.py script from pod to test RBAC for client-readonly-user.
# verify the logs for write operation will show below message 
# --- Write to Feature Store ---
#*** PERMISSION DENIED *** User lacks permission to modify the feature store.

!kubectl exec -n feast -it $(kubectl get pods -n feast -l app=client-user -o jsonpath="{.items[0].metadata.name}") -- python test.py


<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)

--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats
  - driver_hourly_stats_fresh

--- Fetching Historical Features for Training ---
Successfully fetched training historical features:
    driver_id  ... conv_rate_plus_val2
0       1001  ...           10.229559
1       1002  ...           20.697800
2       1003  ...           30.933721

[3 rows x 10 columns]

--- Fetching Historical Features for Batch Scoring ---
Successfully fetched batch scoring historical features:
    driver_id  ... conv_rate_plus_val2
0       1002  ...           20.445888
1       1001  ...           10.815464
2       1003  ...           30.287972

[3 rows x 10 columns]

--- Write to Feature Store ---

*** PERMISSION DENIED *** User lacks permission to modify the feature store.

--- Fetching Online Features ---
Successfully fetched onlin

### Test Unauthorized Feast User 

In [24]:
!kubectl apply -f "client/unauthorized_user_deployment.yaml"

deployment.apps/client-unauthorized-user created


In [25]:
!kubectl exec -n feast -it $(kubectl get pods -n feast -l app=client-unauthorized-user -o jsonpath="{.items[0].metadata.name}") -- python test.py

<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)
/opt/app-root/lib64/python3.11/site-packages/feast/feature_view.py:48: DeprecationWarning: Entity value_type will be mandatory in the next release. Please specify a value_type for entity '__dummy'.
  DUMMY_ENTITY = Entity(

--- List feature views ---
No feature views found. You might not have access or they haven't been created.

--- Fetching Historical Features for Training ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Fetching Historical Features for Batch Scoring ---

*** PERMISSION DENIED *** Cannot fetch historical features.

--- Write to Feature Store ---

*** PERMISSION DENIED *** User lacks permission to modify the feature store.

--- Fetching Online Features ---

*** PERMISSION DENIED *** Cannot fetch online features.

--- Fetching Online Features via Feature Service ---

*** PERMISSION DENIED *** Cannot fetch 

## Test Admin Feast User

In [26]:
!kubectl apply -f "client/admin_user_deployment.yaml"

deployment.apps/client-admin-user created


In [27]:
!kubectl exec -n feast -it $(kubectl get pods -n feast -l app=client-admin -o jsonpath="{.items[0].metadata.name}") -- python test.py


<jemalloc>: MADV_DONTNEED does not work (memset will be used instead)
<jemalloc>: (This is the expected behaviour if you are running under QEMU)

--- List feature views ---
Successfully listed 2 feature views:
  - driver_hourly_stats
  - driver_hourly_stats_fresh

--- Fetching Historical Features for Training ---
Successfully fetched training historical features:
    driver_id  ... conv_rate_plus_val2
0       1001  ...           10.229559
1       1002  ...           20.697800
2       1003  ...           30.933721

[3 rows x 10 columns]

--- Fetching Historical Features for Batch Scoring ---
Successfully fetched batch scoring historical features:
    driver_id  ... conv_rate_plus_val2
0       1002  ...           20.445888
1       1001  ...           10.815464
2       1003  ...           30.287972

[3 rows x 10 columns]

--- Write to Feature Store ---
User has write access to the feature store.

--- Fetching Online Features ---
Successfully fetched online features directly:

acc_rate : [

[Next: Client example on local](./3-client-rbac-test-local.ipynb)